Imports

In [ ]:
from prettytable import PrettyTable
import statsmodels.api as sm
import numpy as np
import pandas as pd
import csv
import math
import numpy as np
import ants
from PIL import Image
import os
import os.path
import glob
import warnings
import shutil
import random
import itertools
from scipy import ndimage
import SimpleITK as sitk
import nibabel as nib
from scipy.ndimage import map_coordinates, gaussian_filter
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from skimage.transform import resize
import cv2
from skimage.measure import label
import scipy.misc
import matplotlib.patches as mpl_patches
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg
from pingouin import mediation_analysis, read_dataset
from statannotations.Annotator import Annotator
import seaborn as sns; sns.set_theme(color_codes=True)
sns.set(style="darkgrid")

Bland Altman plots definition

In [ ]:
def bland_altman_plot_scratch(data1, data2, cortex_label, *args, **kwargs):
    data1     = np.asarray(data1)
    data2     = np.asarray(data2)
    mean      = np.mean([data1, data2], axis=0)
    diff      = data1 - data2                   
    md        = np.mean(diff)                  
    sd        = np.std(diff, axis=0)           
    CI_low    = md - 1.96*sd
    CI_high   = md + 1.96*sd

    df_two_columns['mean'] = mean
    df_two_columns['diff'] = diff

    sns.lmplot(x='mean', y='diff', data=df_two_columns, fit_reg=False, legend=False)

    plt.axhline(md,           color='black', linestyle='--')
    plt.axhline(md + 1.96*sd, color='gray', linestyle='--')
    plt.axhline(md - 1.96*sd, color='gray', linestyle='--')
    
    plt.title(cortex_label, fontsize=20)
    plt.xlabel("Average", fontsize=20)
    plt.ylabel("Difference", fontsize=20)
    plt.ylim(md - 3.5*sd, md + 3.5*sd)

    xOutPlot = np.min(mean) + (np.max(mean)-np.min(mean))*1.14

    plt.text(xOutPlot, md - 1.96*sd, 
        r'-1.96SD:' + "\n" + "%.2f" % CI_low, 
        ha = "center",
        va = "center",
        )
    plt.text(xOutPlot, md + 1.96*sd, 
        r'+1.96SD:' + "\n" + "%.2f" % CI_high, 
        ha = "center",
        va = "center",
        )
    plt.text(xOutPlot, md, 
        r'Mean:' + "\n" + "%.2f" % md, 
        ha = "center",
        va = "center",
        )
    plt.subplots_adjust(right=0.85)
    plt.savefig('/path/to/directory/' + cortex_label + '_BA.png', bbox_inches='tight', dpi=500)
    plt.show()
             

Intraclass correlation coefficient and BA plots between manual and nnU-Net-CRUISE thickness

In [ ]:
######## ICC for the different cortical ROIs

filename = '/path/to/directory/data.csv'
thickness_sheet = pd.read_csv(filename)

######### FOR PLOTS VISUALIZATION ONLY
######### Of course, no covariates

plt.rcParams.update({'font.size': 20})

## Required mappings
cortical_locations_fullname = ['Visual', 'Motor', 'Posterior Cingulate', 'Midfrontal', 'Anterior Cingulate', 'Orbitofrontal',
                               'Superior Temporal', 'Inferior Frontal', 'Anterior Insula', 'Anterior Temporal Pole',
                               'Ventrolateral Temporal', 'Superior Parietal', 'Angular Gyrus', 'ERC', 'BA35', 'PHC']

cortical_locations = ['VIS', 'MOT', 'pcin', 'mf', 'acin', 'orf', 'stemp', 'if',
'antin', 'atempp', 'vlt', 'sp', 'ang', 'ERC', 'BA35', 'PHC']
cortical_locations = map(lambda cortical_locations:cortical_locations.upper(), cortical_locations)
cortical_locations = list(cortical_locations)


icc_list = []
icc_list_for_graphs = []

for cortex in cortical_locations:
    print(">>>>>>>>>>>>> ", cortex)
    
    col_1 = cortex + '_manual'
    col_2 = cortex + '_CRUISE_BINARY_SEGM'
    
    df_two_columns = pd.concat([thickness_sheet['INDD'], thickness_sheet[col_1], thickness_sheet[col_2]], axis=1, keys=['subjects', col_1, col_2])
    df_two_columns = df_two_columns.dropna()

    df_two_columns = df_two_columns.rename(columns={col_1: 'manual', col_2: 'nnunet'})
    df_two_columns_melted = pd.melt(df_two_columns, id_vars=['subjects'], value_vars=['manual', 'nnunet'])
    df_two_columns_melted["value"] = pd.to_numeric(df_two_columns_melted["value"], downcast="float")
    
    icc = pg.intraclass_corr(data=df_two_columns_melted, targets='subjects', raters='variable', ratings='value')
    print(icc.at[5,"ICC"])
    
    icc_list.append(icc)
    icc_list_for_graphs.append(icc.at[5,"ICC"])
    
print(icc_list_for_graphs)
print(np.mean(icc_list_for_graphs))

for cortex, current_path, ICC in zip(cortical_locations, cortical_locations_fullname, icc_list_for_graphs):
    print(cortex)
    
    col_1 = cortex + '_manual'
    col_2 = cortex + '_CRUISE_BINARY_SEGM'

    df_two_columns = pd.concat([thickness_sheet[col_1], thickness_sheet[col_2]],
                               axis=1,
                               keys=[col_1, col_2])
    
    df_two_columns = df_two_columns.loc[~(df_two_columns==0).all(axis=1)]
    df_two_columns = df_two_columns.dropna()

    x = df_two_columns[col_1]
    y = df_two_columns[col_2]

    x = x.to_numpy()
    y = y.to_numpy()

    slope, intercept, r, p_from_linregress, stderr = scipy.stats.linregress(x, y)

    results = smf.glm('df_two_columns[col_2] ~ df_two_columns[col_1]', data=df_two_columns).fit(t_test='True')
    r_spearmanr, p_spearmanr = scipy.stats.spearmanr(x, y)
    
    pg_corr = pg.corr(x=df_two_columns[col_1], y=df_two_columns[col_2], method="spearman")
    pg_corr_P = pg_corr['p-val'][0]
    pg_corr_R = pg_corr['r'][0]

    p_from_ols = results.pvalues[1]
    t_from_ols = results.tvalues[1]

    sns.set(font_scale = 1.5)
    sns.lmplot(x=col_1, y=col_2, data=df_two_columns, fit_reg=False, legend=False)

    hh = sns.regplot(x=col_1, y=col_2, data=df_two_columns, scatter=False)
    hh.set(xlabel=None)
    hh.set(ylabel=None)
    hh.axline(xy1=(1, 1), slope=1, color="black", dashes=(5, 5))

    hh.set_title(current_path)
    plt.tight_layout()

    line = f'r={pg_corr_R:.2f}, p={pg_corr_P:.2e}, ICC={ICC:.2f}'
    handles = [mpl_patches.Rectangle((0, 0), 1, 1, fc="white", ec="white", 
                                 lw=0, alpha=0)]

    hh.legend(handles, [line], loc='upper center', fontsize='medium', 
          fancybox=False, framealpha=0.0, 
          handlelength=0, handletextpad=0)
    sns.set(rc = {'figure.figsize':(15, 8)})
    
    plt.show()
    #plt.savefig('/path/to/directory/' + cortex + '.png', bbox_inches='tight', dpi=300)


for cortex, cortex_label in zip(cortical_locations, cortical_locations_fullname):
    print(cortex)
    
    col_1 = cortex + '_manual'
    col_2 = cortex + '_CRUISE_BINARY_SEGM'
    
    df_two_columns = pd.concat([thickness_sheet[col_1], thickness_sheet[col_2]],
                               axis=1,
                               keys=[col_1, col_2])
    df_two_columns = df_two_columns.dropna()
    
    x = df_two_columns[col_1]
    y = df_two_columns[col_2]

    x = x.to_numpy()
    y = y.to_numpy()
    
    bland_altman_plot_scratch(x, y, cortex_label)
    

Correlation between nnU-Net-CRUISE thickness and Abeta, BRAAK, CERAD

In [ ]:
## cortical locations and pathology mappings
cortical_dots = ['VIS', 'MOT', 'PCIN', 'MF', 'ACIN', 'ORF', 'STEMP', 'IF', 'ANTIN', 'ATEMPP', 'VLT', 'SP', 'ANG', 'ERC', 'BA35', 'PHC']
path_mappings = ['OC', 'MC', 'CING', 'MF', 'CING', 'OFC', 'SMT', 'MF', 'MF', 'AMYG', 'EC', 'ANG', 'ANG', 'EC', 'EC', 'CS']

suffix = '_CRUISE_BINARY_SEGM'
cortical_dots = list(map(lambda x: x + suffix, cortical_dots))
print(cortical_dots)

cortical_locations_fullname = ['Visual', 'Motor', 'Posterior Cingulate', 'Midfrontal', 'Anterior Cingulate', 'Orbitofrontal',
                       'Superior Temporal', 'Inferior Frontal', 'Anterior Insula', 'Anterior Temporal Pole',
                       'Ventrolateral Temporal', 'Superior Parietal', 'Angular Gyrus', 'ERC', 'BA35', 'PHC']
    
## covariates
covar=['AGEATDEATH', 'SEX', 'PMI']

## independent variable
curr_variable = 'ABETA'
curr_variable = 'BRAAK06'
curr_variable = 'CERAD'


print(len(cortical_dots), len(cortical_locations_fullname))

all_p_values = []
all_stats = []
for cortex, current_path in zip(cortical_dots, cortical_locations_fullname):
    try:
        print(cortex, current_path, curr_variable)  
        columns_use = pd.concat([thickness_sheet[cortex], thickness_sheet[curr_variable],
                                thickness_sheet['AGEATDEATH'], thickness_sheet['SEX'], thickness_sheet['HEMIS'], thickness_sheet['PMI']],
                                       axis=1,
                                       keys=[cortex, curr_variable, 'AGEATDEATH', 'SEX', 'HEMIS', 'PMI'])
        
        columns_use = columns_use.loc[~(columns_use==0).all(axis=1)]
        columns_use = columns_use.dropna()
        
        values = pg.partial_corr(data=columns_use, y=cortex, x=curr_variable, covar=covar, method='spearman', alternative='less')

        all_p_values.append(values['p-val'][0])
        all_stats.append([cortex, values['n'][0], values['r'][0].round(3), list(values['CI95%'][0]), values['p-val'][0].round(4)])
        
        ##### plots
        pg_corr = pg.corr(x=columns_use[curr_variable], y=columns_use[cortex], method="spearman")
        pg_corr_P = pg_corr['p-val'][0]
        pg_corr_R = pg_corr['r'][0]
        
        rho_plot = values['r'][0]
        p_plot = values['p-val'][0]
        
        print(pg_corr_R, pg_corr_P)
        print(rho_plot, p_plot)
        
        sns.set(font_scale = 2)
        hh = sns.regplot(x=curr_variable, y=cortex, data=columns_use, scatter=True)
        hh.set(xlabel=None)
        hh.set(ylabel=None)
        hh.set_title(current_path)

        line = f'rho={rho_plot:.2f}, p={p_plot:.2e}'
        handles = [mpl_patches.Rectangle((0, 0), 1, 1, fc="white", ec="white", 
                                     lw=0, alpha=0)]

        hh.legend(handles, [line], loc='upper right', fontsize='medium', 
              fancybox=False, framealpha=0.0, 
              handlelength=0, handletextpad=0)
        sns.set(rc = {'figure.figsize':(15,8)})

        plt.savefig('/path/to/directory/' + cortex + '_' + current_path +'.png', bbox_inches='tight', dpi=300)
        plt.clf()
    except:
        continue

print(all_stats)
print("UNCORREDTED p-values:  ", all_p_values, len(all_p_values))
reject, corrected_all_p_values = pg.multicomp(all_p_values, method='bonf')
print("After Bonferroni CORREDTED p-values:  ", reject, corrected_all_p_values, len(corrected_all_p_values))

#### append the corrected_all_p_values to the all_stats list
corrected_all_p_values = corrected_all_p_values.round(4)
corrected_all_p_values = corrected_all_p_values.tolist()

pretty_table = PrettyTable()
pretty_table.field_names = ['cortex_label', 'N subjs', 'rho', 'CI', 'p-value', 'p-corr']

all_rows_as_list = []
for a, b in zip(all_stats, corrected_all_p_values):
    c = a + [b]
    print(c, type(c))
    pretty_table.add_row(c)
    all_rows_as_list.append(c)
    

with open("/path/to/directory/to_save.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(all_rows_as_list)

table_txt = pretty_table.get_string()
with open('/path/to/directory/to_save.txt','w') as file:
    file.write(table_txt)

print(pretty_table)


Correlation between nnU-Net-CRUISE thickness and Tau, Neuronal loss

In [ ]:
cortical_locations_fullname = ['Visual', 'Posterior Cingulate', 'Midfrontal', 'Anterior Cingulate',
                       'Superior Temporal', 'Inferior Frontal', 'Anterior Insula', 'Anterior Temporal Pole',
                       'Ventrolateral Temporal', 'Superior Parietal', 'Angular Gyrus', 'ERC', 'BA35', 'PHC']
    
## cortical locations and pathology mappings NOTICE THAT THESE ARE 14 REGIONS
cortical_dots = ['VIS', 'PCIN', 'MF', 'ACIN', 'STEMP', 'IF', 'ANTIN', 'ATEMPP', 'VLT', 'SP', 'ANG', 'ERC', 'BA35', 'PHC']
path_mappings = ['OC', 'CING', 'MF', 'CING', 'SMT', 'MF', 'MF', 'AMYG', 'EC', 'ANG', 'ANG', 'EC', 'EC', 'CS']

suffix = '_CRUISE_BINARY_SEGM'
cortical_dots = list(map(lambda x: x + suffix, cortical_dots))
print(cortical_dots)

#### For the MTL EC_CS_DG pathology
#path_mappings = ['EC_CS_DG']*14

## covariates
covar=['AGEATDEATH', 'SEX', 'PMI']

## independent variable
ind_variable = 'NEURONLOSS'
ind_variable = 'TAU'

all_p_values = []
all_stats = []
for cortex, path_map, current_path in zip(cortical_dots, path_mappings, cortical_locations_fullname):
    try:
        curr_variable = path_map + ind_variable                 
        columns_use = pd.concat([thickness_sheet[cortex], thickness_sheet['AGEATDEATH'], thickness_sheet['SEX'], thickness_sheet['HEMIS'], thickness_sheet['PMI'],
                                thickness_sheet[path_map+'NEURONLOSS'], thickness_sheet[path_map+'TAU']],
                                axis=1,
                                keys=[cortex, 'AGEATDEATH', 'SEX', 'HEMIS', 'PMI',
                                      path_map+'NEURONLOSS', path_map+'TAU'])
        
        columns_use = columns_use.loc[~(columns_use==0).all(axis=1)]
        columns_use = columns_use.dropna()
        
        values = pg.partial_corr(data=columns_use, y=cortex, x=curr_variable, covar=covar, method='spearman', alternative='less')
        
        all_p_values.append(values['p-val'][0])
        all_stats.append([cortex, values['n'][0], values['r'][0].round(3), list(values['CI95%'][0]), values['p-val'][0].round(4)])
        
        ##### plots
        pg_corr = pg.corr(x=columns_use[curr_variable], y=columns_use[cortex], method="spearman")
        pg_corr_P = pg_corr['p-val'][0]
        pg_corr_R = pg_corr['r'][0]
        
        rho_plot = values['r'][0]
        p_plot = values['p-val'][0]
        
        sns.set(font_scale = 2)
        hh = sns.regplot(x=curr_variable, y=cortex, data=columns_use, scatter=True)
        hh.set(xlabel=None)
        hh.set(ylabel=None)
        hh.set_title(current_path)

        line = f'rho={rho_plot:.2f}, p={p_plot:.2e}'
        handles = [mpl_patches.Rectangle((0, 0), 1, 1, fc="white", ec="white", 
                                     lw=0, alpha=0)]

        hh.legend(handles, [line], loc='upper right', fontsize='medium', 
              fancybox=False, framealpha=0.0, 
              handlelength=0, handletextpad=0)
        sns.set(rc = {'figure.figsize':(15,8)})

        plt.savefig('/path/to/directory/' + cortex + '_' + current_path +'.png', bbox_inches='tight', dpi=300)
        plt.clf()        
        
    except:
        continue
    

#print("UNCORREDTED p-values:  ", all_p_values, len(all_p_values))
reject, corrected_all_p_values = pg.multicomp(all_p_values, method='bonf')
#print("After Bonferroni CORREDTED p-values:  ", reject, corrected_all_p_values, len(corrected_all_p_values))

#### append the corrected_all_p_values to the all_stats list
corrected_all_p_values = corrected_all_p_values.round(4)
corrected_all_p_values = corrected_all_p_values.tolist()

pretty_table = PrettyTable()
pretty_table.field_names = ['cortex_label', 'N subjs', 'rho', 'CI', 'p-value', 'p-corr']
    
all_rows_as_list = []
for a, b in zip(all_stats, corrected_all_p_values):
    c = a + [b]
    print(c, type(c))
    pretty_table.add_row(c)
    all_rows_as_list.append(c)
    
with open("/path/to/directory//this_goes_in_paper/nloss_mtl_automated.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(all_rows_as_list)    
    
    
table_txt = pretty_table.get_string()
with open('/path/to/directory//this_goes_in_paper/nloss_mtl_automated.txt','w') as file:
    file.write(table_txt)

print(pretty_table)


Correlation between WMH burden and nnU-Net-CRUISE thickness, subcortical volumetry

In [ ]:
cortical_locations_fullname = ['Visual', 'Motor', 'Posterior Cingulate', 'Midfrontal', 'Anterior Cingulate', 'Orbitofrontal',
                       'Superior Temporal', 'Inferior Frontal', 'Anterior Insula', 'Anterior Temporal Pole',
                       'Ventrolateral Temporal', 'Superior Parietal', 'Angular Gyrus', 'ERC', 'BA35', 'PHC']
    
## cortical locations
cortical_dots = ['VIS', 'MOT', 'PCIN', 'MF', 'ACIN', 'ORF', 'STEMP', 'IF', 'ANTIN', 'ATEMPP', 'VLT', 'SP', 'ANG', 'ERC', 'BA35', 'PHC']

suffix = '_CRUISE_BINARY_SEGM'
cortical_dots = list(map(lambda x: x + suffix, cortical_dots))
cortical_dots.extend(['CAUDATE', 'GLOBUS PALLIDUS', 'PUTAMEN', 'THALAMUS'])
print(cortical_dots)

cortical_locations_fullname.extend(['Caudate', 'Globus Pallidus', 'Putamen', 'Thalamus'])
print(cortical_locations_fullname)

## covariates
covar=['AGEATDEATH', 'SEX', 'PMI']

curr_variable = 'WMH_WM'
all_p_values = []
all_stats = []
for cortex, current_path in zip(cortical_dots, cortical_locations_fullname):
    #print(cortex, curr_variable)                   
    columns_use = pd.concat([thickness_sheet[cortex], thickness_sheet[curr_variable],
                            thickness_sheet['AGEATDEATH'], thickness_sheet['SEX'], thickness_sheet['HEMIS'], thickness_sheet['PMI'], thickness_sheet['ABETA']],
                                   axis=1,
                                   keys=[cortex, curr_variable, 'AGEATDEATH', 'SEX', 'HEMIS', 'PMI', 'ABETA'])
    
    columns_use = columns_use.loc[~(columns_use==0).all(axis=1)]
    columns_use = columns_use.dropna()

    values = pg.partial_corr(data=columns_use, y=cortex, x=curr_variable, covar=covar, method='spearman', alternative='less')
    all_p_values.append(values['p-val'][0])
    all_stats.append([cortex, values['n'][0], values['r'][0].round(3), list(values['CI95%'][0]), values['p-val'][0].round(4)])

    ##### plots
    pg_corr = pg.corr(x=columns_use[curr_variable], y=columns_use[cortex], method="spearman")
    pg_corr_P = pg_corr['p-val'][0]
    pg_corr_R = pg_corr['r'][0]

    rho_plot = values['r'][0]
    p_plot = values['p-val'][0]

    sns.set(font_scale = 2)
    hh = sns.regplot(x=curr_variable, y=cortex, data=columns_use, scatter=True)
    hh.set(xlabel=None)
    hh.set(ylabel=None)
    hh.set_title(current_path)

    #hh.set_xlim(0, 3.5)
    #hh.set_ylim(0, 6.5)

    line = f'rho={rho_plot:.2f}, p={p_plot:.2e}'
    handles = [mpl_patches.Rectangle((0, 0), 1, 1, fc="white", ec="white", 
                                 lw=0, alpha=0)]

    hh.legend(handles, [line], loc='upper right', fontsize='medium', 
          fancybox=False, framealpha=0.0, 
          handlelength=0, handletextpad=0)
    sns.set(rc = {'figure.figsize':(15,8)})

    plt.savefig('/path/to/directory/' + cortex + '_' + current_path +'.png', bbox_inches='tight', dpi=300)
    plt.clf()  
    
print(all_stats)
print("UNCORREDTED p-values:  ", all_p_values, len(all_p_values))
reject, corrected_all_p_values = pg.multicomp(all_p_values, method='bonf')
print("After Bonferroni CORREDTED p-values:  ", reject, corrected_all_p_values, len(corrected_all_p_values))

#### append the corrected_all_p_values to the all_stats list
corrected_all_p_values = corrected_all_p_values.round(4)
corrected_all_p_values = corrected_all_p_values.tolist()

pretty_table = PrettyTable()
pretty_table.field_names = ['cortex_label', 'N subjs', 'rho', 'CI', 'p-value', 'p-corr']

all_rows_as_list = []
for a, b in zip(all_stats, corrected_all_p_values):
    c = a + [b]
    print(c, type(c))
    pretty_table.add_row(c)
    all_rows_as_list.append(c)
    
with open("/path/to/directory/wmh.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(all_rows_as_list)    
       
table_txt = pretty_table.get_string()
with open('/path/to/directory/wmh.txt','w') as file:
    file.write(table_txt)

print(pretty_table)

Box plots for the subcortical volumtery

In [ ]:
print(thickness_sheet['NPDX_SUBCORTICAL'].value_counts())

list_columns = thickness_sheet.columns
states_order = list(thickness_sheet.NPDX_SUBCORTICAL.unique())
print(states_order)

pairs = []
for pair in itertools.combinations(states_order,2):
    pairs.append(pair)
print(pairs)
states_palette = sns.color_palette("Pastel2")


#####
structure = 'CAUDATE'
state_plot_params = {
    'data': thickness_sheet,
    'x': 'NPDX_SUBCORTICAL',
    'y': structure,
    'order': states_order,
    'palette': states_palette
}

with sns.plotting_context('notebook', font_scale = 0.5):
    sns.set(font_scale =1.0)
    
    ax = sns.boxplot(data=thickness_sheet, x='NPDX_SUBCORTICAL', y=structure, order=states_order, showfliers=False)
    
    ax.tick_params(axis='x', rotation=90, labelsize=20)
    to_plot = 'Caudate ($mm^{3}$)'
    ax.set_ylabel(to_plot, fontsize =20)

    annotator = Annotator(ax=ax, pairs=pairs, **state_plot_params)
    annotator.configure(test='t-test_ind', comparisons_correction="Bonferroni").apply_and_annotate()
    plt.savefig('/path/to/directory/' + structure + '.png', bbox_inches='tight', dpi=300)
    

In [ ]:
structure = 'PUTAMEN'
with sns.plotting_context('notebook', font_scale = 1.4):
    sns.set(font_scale =1.0)
    
    ax = sns.boxplot(data=thickness_sheet, x='NPDX_SUBCORTICAL', y=structure, order=states_order, showfliers=False)    
    ax.tick_params(axis='x', rotation=90, labelsize=20)
    to_plot = 'Putamen ($mm^{3}$)'
    ax.set_ylabel(to_plot, fontsize =20)
    
    annotator = Annotator(ax=ax, pairs=pairs, **state_plot_params)
    annotator.configure(test='t-test_ind', comparisons_correction="Bonferroni").apply_and_annotate()
    plt.savefig('/path/to/directory/' + structure + '.png', bbox_inches='tight', dpi=300)



In [ ]:
structure = 'THALAMUS'
with sns.plotting_context('notebook', font_scale = 1.4):
    sns.set(font_scale =1.0)
    
    ax = sns.boxplot(data=thickness_sheet, x='NPDX_SUBCORTICAL', y=structure, order=states_order, showfliers=False)
    ax.tick_params(axis='x', rotation=90, labelsize=20)
    to_plot = 'Thalamus ($mm^{3}$)'
    ax.set_ylabel(to_plot, fontsize =20)

    annotator = Annotator(ax=ax, pairs=pairs, **state_plot_params)
    annotator.configure(test='t-test_ind', comparisons_correction="Bonferroni").apply_and_annotate()
    plt.savefig('/path/to/directory/' + structure + '.png', bbox_inches='tight', dpi=300)


In [ ]:
structure = 'GLOBUS PALLIDUS'
with sns.plotting_context('notebook', font_scale = 1.4):
    sns.set(font_scale =1.0)

    ax = sns.boxplot(data=thickness_sheet, x='NPDX_SUBCORTICAL', y=structure, order=states_order, showfliers=False)
    ax.tick_params(axis='x', rotation=90, labelsize=20)
    to_plot = 'Globus Pallidus ($mm^{3}$)'
    ax.set_ylabel(to_plot, fontsize =20)
    
    annotator = Annotator(ax=ax, pairs=pairs, **state_plot_params)
    annotator.configure(test='t-test_ind', comparisons_correction="Bonferroni").apply_and_annotate()

    plt.savefig('/path/to/directory/' + structure + '.png', bbox_inches='tight', dpi=300)

    